In [2]:
%matplotlib inline
import csv
import pandas as pd
import scipy.sparse as sc
from helpers import *
from sgd import *
from als import *
from sklearn import decomposition
from scipy.optimize import minimize
from models import *
%load_ext autoreload
%autoreload 2

In [3]:
ratings = load_data('data_train.csv')

number of items: 10000, number of users: 1000


In [4]:
num_items_per_user = np.array((ratings != 0).sum(axis=0)).flatten()
num_users_per_item = np.array((ratings != 0).sum(axis=1).T).flatten()

In [5]:
_,train, test = split_data(ratings, num_items_per_user, num_users_per_item, 10, 0.1)

the shape of original ratings. (# of row, # of col): (10000, 1000)
the shape of valid ratings. (# of row, # of col): (9990, 999)
Total number of nonzero elements in origial data:1176952
Total number of nonzero elements in train data:1065253
Total number of nonzero elements in test data:111620


In [12]:
nnz_row, nnz_col = test.nonzero()
nnz_test = list(zip(nnz_row, nnz_col))

In [7]:
predict_global = baseline_global_mean(train, test)
predict_global

test RMSE of baseline using the global mean: [[ 1.12152228]].


3.8581107023810133

In [8]:
print(train.shape)
predict_SGD = matrix_factorization_SGD(train, test)

(9990, 999)
learn the matrix factorization using SGD...
iter: 0, RMSE on training set: 0.9975633866643219.
iter: 1, RMSE on training set: 0.98399317335416.
iter: 2, RMSE on training set: 0.977559146774514.
iter: 3, RMSE on training set: 0.973302024837026.
iter: 4, RMSE on training set: 0.9697063269801259.
iter: 5, RMSE on training set: 0.9684855523087732.
iter: 6, RMSE on training set: 0.9652848076933224.
iter: 7, RMSE on training set: 0.9637783260052316.
iter: 8, RMSE on training set: 0.9623884166935112.
iter: 9, RMSE on training set: 0.9610078532191024.
iter: 10, RMSE on training set: 0.9598546211932161.
iter: 11, RMSE on training set: 0.9594652752124836.
iter: 12, RMSE on training set: 0.9587680183976469.
iter: 13, RMSE on training set: 0.9583004685120122.
iter: 14, RMSE on training set: 0.9579096792154936.
iter: 15, RMSE on training set: 0.9575326248832436.
iter: 16, RMSE on training set: 0.9572602341203271.
iter: 17, RMSE on training set: 0.9569018531988474.
iter: 18, RMSE on trai

In [9]:
predict_ALS = ALS(train,test,3, 0.2, 0.9)


start the ALS algorithm...
RMSE on training set: 0.9683854696262746.
RMSE on training set: 0.9636641888328061.
RMSE on training set: 0.9622033086723247.
RMSE on training set: 0.9610708830609189.
RMSE on training set: 0.9600981287338146.
RMSE on training set: 0.9593451001369316.
RMSE on training set: 0.9588289719349584.
RMSE on training set: 0.9585034014048226.
RMSE on training set: 0.9583079650772224.
RMSE on training set: 0.958193546877028.
RMSE on training set: 0.9581268728266509.
test RMSE after running ALS: 0.9940086593042198.


In [13]:
def const(x):
    return x.sum() - 1
constraint = {'type':'eq', 'fun': const}

def sum_ratings(weights):
    mix_prediction = weights[0] * predict_SGD + weights[1] * predict_ALS + weights[2] * predict_global
    return compute_mix_error(test, nz=nnz_test, prediction=mix_prediction)


In [16]:
result = minimize(fun=sum_ratings, x0=[0.33, 0.33, 0.33])

In [17]:
result

      fun: 0.98300020764034
 hess_inv: array([[  2.58353985e+01,  -2.17302627e+01,  -3.47105888e+00],
       [ -2.17302627e+01,   2.12120474e+01,  -1.52044929e-02],
       [ -3.47105888e+00,  -1.52044929e-02,   3.46331025e+00]])
      jac: array([ -5.33461571e-06,  -5.58793545e-06,  -5.53578138e-06])
  message: 'Optimization terminated successfully.'
     nfev: 80
      nit: 14
     njev: 16
   status: 0
  success: True
        x: array([ 0.70057756,  0.31197472,  0.00211984])

In [18]:
predict_global = baseline_global_mean(ratings, None)
predict_SGD = matrix_factorization_SGD(ratings,None)
predict_ALS = ALS(ratings,None, 3, 0.2, 0.9)

learn the matrix factorization using SGD...
iter: 0, RMSE on training set: 0.997560255008501.
iter: 1, RMSE on training set: 0.9842293566398914.
iter: 2, RMSE on training set: 0.9780387061695399.
iter: 3, RMSE on training set: 0.9723799399467854.
iter: 4, RMSE on training set: 0.969301806280314.
iter: 5, RMSE on training set: 0.9667096170339626.
iter: 6, RMSE on training set: 0.964171308803295.
iter: 7, RMSE on training set: 0.9629057008668301.
iter: 8, RMSE on training set: 0.9620649935787686.
iter: 9, RMSE on training set: 0.9603911798400415.
iter: 10, RMSE on training set: 0.959463560281415.
iter: 11, RMSE on training set: 0.9585125420480426.
iter: 12, RMSE on training set: 0.9581533270460627.
iter: 13, RMSE on training set: 0.957696042239014.
iter: 14, RMSE on training set: 0.9571881375998007.
iter: 15, RMSE on training set: 0.9567520143874916.
iter: 16, RMSE on training set: 0.9566212727648964.
iter: 17, RMSE on training set: 0.9561960422110439.
iter: 18, RMSE on training set: 0.9

In [19]:
final_predict = result.x[0] * predict_SGD + result.x[1] * predict_ALS + result.x[2] * predict_global

In [20]:
final_predict[final_predict<1] = 1
final_predict[final_predict>5] = 5

In [21]:
create_csv_submissions(final_predict)

In [22]:
final_predict.shape

(10000, 1000)